**Einlesen der Daten**

In [ ]:
import pandas as pd

df = pd.read_csv("trainingsdaten_ohne_vibration.csv", encoding = "ISO-8859-1")
df_test = pd.read_csv("testdaten_mit_vibration.csv", encoding = "ISO-8859-1")

**Trainingsdaten anschauen**

In [ ]:
df.head()

**Testdaten anschauen**

In [ ]:
df_test.head()

**Deskriptive Statistik (Trainingsdaten)**

In [ ]:
df.describe()

**Deskriptive Statistik (Testdaten)**

In [ ]:
df_test.describe()

**Daten visualisieren**

In [ ]:
import matplotlib.pyplot as plt

df.plot(figsize=(10,10),subplots=True, layout=(3,2), title="Daten", legend=True)
plt.show()

In [ ]:
df_test.plot(figsize=(10,10),subplots=True, layout=(3,2), title="Daten", legend=True)
plt.show()

**Datenvorverarbeitung**

In [ ]:
# Time löschen + 2 Spalten, die keine Information enthalten
df = df.drop(['Time', 'NumberOverload', 'NumberUnderloads'], 1)
df_test = df_test.drop(['Time', 'NumberOverload', 'NumberUnderloads'], 1)

**Test- und Trainingsdaten festlegen**

In [ ]:
X_train = df
X_test = df_test[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].copy()
y_test = df_test[['VibrationMotorOn']].copy()

**Modell trainieren**

In [ ]:
from sklearn.svm import OneClassSVM

# Modell erstellen
clf = OneClassSVM(random_state=42, nu = 0.15, kernel = "rbf", gamma = 0.4)

# Modell an Trainingsdaten anpassen ("fit")
clf.fit(X_train)

pred = pd.DataFrame(columns = ["Anomaly"])

# Vorhersage des Modells
pred['Anomaly'] = pd.Series(clf.predict(X_test[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']]))

# Vorhersage des Modells auf 1/0 recodieren
pred['Anomaly'] = pred['Anomaly'].map( {1: 0, -1: 1} )

**Metriken ausgeben**

In [ ]:
from sklearn import metrics

print ("Accuracy: ")
print (metrics.accuracy_score(y_test, pred))
print ("Classification Report:")
print (metrics.classification_report(y_test, pred))
print ("Confusion Matrix:")
print (metrics.confusion_matrix(y_test, pred))  

**Daten zusammenführen für Visualisierung**

In [ ]:
complete_data = X_test.copy()
complete_data['VibrationMotorOn'] = y_test['VibrationMotorOn']
complete_data['Anomaly'] = pred.values

**Farben für Visualisierung einfügen**

In [ ]:
complete_data["color"] = 0
for idx,row in complete_data.iterrows():
    label_true = complete_data.loc[idx,"VibrationMotorOn"]
    label_pred = complete_data.loc[idx,"Anomaly"]
    if  label_true == True:
        if  label_pred == True:
            complete_data.loc[idx,"color"] = "blue"
        else:
            complete_data.loc[idx,"color"] = "green"
    else:
        if label_pred == False:
            complete_data.loc[idx,"color"] = "red"            
        else:
            complete_data.loc[idx,"color"] = "yellow"

**Visualisierung der Konfusionsmatrix**

In [ ]:
fig, ax = plt.subplots()
a = complete_data.copy()

ax.scatter(a.loc[a.color == 'blue'].index,a.loc[a.color == 'blue',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='blue', label='True positive')
ax.scatter(a.loc[a.color == 'green'].index,a.loc[a.color == 'green',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='green', label='False negative')
ax.scatter(a.loc[a.color == 'red'].index,a.loc[a.color == 'red',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='red', label='True negative')
ax.scatter(a.loc[a.color == 'yellow'].index,a.loc[a.color == 'yellow',['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].mean(axis=1), color='yellow', label='False positive')

plt.legend()
plt.show()